In [8]:
try:
    %tensorflow_version 2.x
except Exception:
    pass

In [15]:
from __future__ import absolute_import, division, print_function, unicode_literals, unicode_literals

# tensorflow와 tf.keras를 import
import tensorflow as tf
from tensorflow import keras

import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import random

print(tf.__version__)

2.2.0


In [16]:
class_names = os.listdir("../feature/feature_mfcc/")
class_names.sort()

print(class_names)

['2017019740001_kwakjuheon', '2017019740002_kimjiho', '2017019740003_songyejin', '2017019740004_parksohui', '2017019740005_moonyeonwoo', '2017019740006_kangsubin', '2017019740007_leekyeongeun', '2017019740008_chochaeyeon', '2017019740009_shindonghwan', '2017019740010_kwakmirae', '2017019740011_kimhyeryeong', '2017019740012_chaeminjoon', '2017019740013_kwakyiheon', '2017019740014_leesumin', '2017019740015_choihayoung', '2017019740016_kangyeseo', '2017019740017_kwakmihyang', '2017019740018_eundano', '2017019740019_limjinju', '2017019770001_kwonyuna', '2017019770002_kwoneunkyung', '2017019770003_johaesu', '2017019770004_leebyeongjin', '2017019770005_hyeonsanghyeok', '2017019770006_jennie', '2017019770007_simseungmin', '2017019770008_parksomi', '2017019770009_yuminji', '2017019770010_parksojin', '2017019770011_kwonyeonwoo', '2017019770012_ladakyeong', '2017019770013_janghyomin', '2017019770014_parkeunbi', '2017019770015_parkdayeung', '2017019770016_parkjongae', '2017019770017_hansohee', '2

In [17]:
x_list = []
y_list = []

for i in range(0,len(class_names)):
    files=os.listdir("../feature/feature_mfcc/"+class_names[i])
    files = [file for file in files if file.endswith(".npy")]
    print(class_names[i].split('_')[0])

    for j in range(0, len(files)):
        print(files[j], j)
        x = np.load("../feature/feature_mfcc/"+class_names[i]+"/"+files[j])
        x_list.append(x)
        y_list.append(i)


2017019740001
2017019740001_kwakjuheon_4.npy 0
2017019740001_kwakjuheon_1.npy 1
2017019740001_kwakjuheon_0.npy 2
2017019740001_kwakjuheon_2.npy 3
2017019740001_kwakjuheon_3.npy 4
2017019740002
2017019740002_kimjiho_3.npy 0
2017019740002_kimjiho_1.npy 1
2017019740002_kimjiho_2.npy 2
2017019740002_kimjiho_4.npy 3
2017019740002_kimjiho_0.npy 4
2017019740003
2017019740003_songyejin_2.npy 0
2017019740003_songyejin_3.npy 1
2017019740003_songyejin_4.npy 2
2017019740003_songyejin_1.npy 3
2017019740003_songyejin_0.npy 4
2017019740004
2017019740004_parksohui_2.npy 0
2017019740004_parksohui_1.npy 1
2017019740004_parksohui_3.npy 2
2017019740004_parksohui_0.npy 3
2017019740004_parksohui_4.npy 4
2017019740005
2017019740005_moonyeonwoo_3.npy 0
2017019740005_moonyeonwoo_2.npy 1
2017019740005_moonyeonwoo_0.npy 2
2017019740005_moonyeonwoo_4.npy 3
2017019740005_moonyeonwoo_1.npy 4
2017019740006
2017019740006_kangsubin_3.npy 0
2017019740006_kangsubin_1.npy 1
2017019740006_kangsubin_4.npy 2
2017019740006_k

In [18]:
from keras.models import Sequential
import tensorflow as tf
from keras.layers import Dense 
from keras.models import Model
from keras.layers import Dense, Embedding, Input, Activation, Dropout, Reshape, concatenate
from keras import regularizers
from keras.wrappers.scikit_learn import KerasClassifier 
import sklearn
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import cross_validate, cross_val_predict
from keras.layers import Conv1D, Conv2D
from keras.layers import MaxPooling1D, MaxPooling3D, MaxPooling2D
from keras.layers.core import Flatten
from keras.optimizers import Adadelta
import numpy

# Function to create model, required for KerasClassifier 
def create_model(X_train, X_val, y_train, y_val): 
    # create model 
    conv21 = keras.models.Sequential()
    conv21.add(keras.layers.Conv2D(32, (3, 3), activation="relu", padding="same", input_shape=(X_train.shape[1], X_train.shape[2], 1)))
    # conv2.add(MaxPooling2D((2, 2)))
    conv21.add(keras.layers.Conv2D(64, (3, 3), activation="relu", padding="same"))
    conv21.add(keras.layers.MaxPooling2D((2, 2)))
    conv21.add(keras.layers.Dropout(0.25))
    conv21.add(keras.layers.Flatten())
    conv21.add(keras.layers.Dense(128, activation="relu"))
    conv21.add(keras.layers.Dropout(0.25))
    conv21.add(keras.layers.Dense(y_val.shape[1], activation="softmax"))
    conv21.summary()
    conv21.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#     print("X_val : ", X_val.shape)
#     print("y_val : ", y_val.shape)
    val_acc = conv21.evaluate(X_val, y_val, batch_size=16, verbose = 1)
    history = conv21.fit(X_val, y_val, batch_size=4, epochs=50, verbose=1, validation_data=(x_val, y_val_encoded))
    return conv21, val_acc

In [19]:
n_folds = 10
cv_scores, model_history = list(), list()
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

x_train = np.array(x_list)
y_train = np.array(y_list)

for i in range(4):
    print("\n================")
    print("| Number : " , i, " |")
    print("================")
    # split data    
    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, train_size=0.25, test_size=0.33, random_state = 32, shuffle=True)
#     x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.33, random_state=50, shuffle=True)
    # evaluate model
    print("x_train: ", x_train.shape)
    print("y_train: ", y_train.shape)
    print("x_val: ", x_val.shape)
    print("y_val: ", y_val.shape)

    print("-----------------------")
    y_train_encoded = tf.keras.utils.to_categorical(y_train)
    y_val_encoded = tf.keras.utils.to_categorical(y_val)
    
    
    x_train = x_train.reshape(-1, x_train.shape[1], x_train.shape[2], 1)
    x_val   = x_val.reshape(-1, x_val.shape[1], x_val.shape[2], 1)


    x_train = x_train - np.min(x_train)

    x_train = x_train / np.max(x_train)

    x_val = x_val - np.min(x_val)
    x_val = x_val / np.max(x_val)
    print("y_train_encoded:", y_train_encoded.shape)
    print("y_val_encoded:", y_val_encoded.shape)
    print("-----------------------")
    
    model, val_acc = create_model(x_train, x_val, y_train_encoded, y_val_encoded)
    print("x_train: ", x_train.shape)
    print("y_train: ", y_train.shape)
    print("x_val: ", x_val.shape)
    print("y_val: ", y_val.shape)

    
    print("val_acc : ", val_acc)
    cv_scores.append(val_acc)
    model_history.append(model)
    
    
    
print('Estimated Accuracy %.3f (%.3f)' % (np.mean(cv_scores), np.std(cv_scores)))


| Number :  0  |
x_train:  (67, 469, 12)
y_train:  (67,)
x_val:  (90, 469, 12)
y_val:  (90,)
-----------------------
y_train_encoded: (67, 51)
y_val_encoded: (90, 54)
-----------------------
Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 469, 12, 32)       320       
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 469, 12, 64)       18496     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 234, 6, 64)        0         
_________________________________________________________________
dropout_16 (Dropout)         (None, 234, 6, 64)        0         
_________________________________________________________________
flatten_8 (Flatten)          (None, 89856)             0         
____________________________________________________________

23/23 [==============================] - 1s 50ms/step - loss: 0.0734 - accuracy: 0.9778 - val_loss: 0.0012 - val_accuracy: 1.0000
Epoch 47/50
23/23 [==============================] - 1s 49ms/step - loss: 0.0689 - accuracy: 0.9778 - val_loss: 5.3253e-04 - val_accuracy: 1.0000
Epoch 48/50
23/23 [==============================] - 1s 49ms/step - loss: 0.0764 - accuracy: 0.9778 - val_loss: 3.2872e-04 - val_accuracy: 1.0000
Epoch 49/50
23/23 [==============================] - 1s 49ms/step - loss: 0.0417 - accuracy: 0.9889 - val_loss: 2.9875e-04 - val_accuracy: 1.0000
Epoch 50/50
23/23 [==============================] - 1s 50ms/step - loss: 0.0666 - accuracy: 0.9778 - val_loss: 1.9824e-04 - val_accuracy: 1.0000
x_train:  (67, 469, 12, 1)
y_train:  (67,)
x_val:  (90, 469, 12, 1)
y_val:  (90,)
val_acc :  [3.99025821685791, 0.011111111380159855]

| Number :  1  |
x_train:  (16, 469, 12, 1)
y_train:  (16,)
x_val:  (23, 469, 12, 1)
y_val:  (23,)
-----------------------
y_train_encoded: (16, 45)
y_

Epoch 41/50
6/6 [==============================] - 0s 47ms/step - loss: 0.0197 - accuracy: 1.0000 - val_loss: 2.6074e-04 - val_accuracy: 1.0000
Epoch 42/50
6/6 [==============================] - 0s 46ms/step - loss: 0.0031 - accuracy: 1.0000 - val_loss: 2.6687e-04 - val_accuracy: 1.0000
Epoch 43/50
6/6 [==============================] - 0s 45ms/step - loss: 0.0317 - accuracy: 1.0000 - val_loss: 2.6258e-04 - val_accuracy: 1.0000
Epoch 44/50
6/6 [==============================] - 0s 45ms/step - loss: 0.0033 - accuracy: 1.0000 - val_loss: 2.8099e-04 - val_accuracy: 1.0000
Epoch 45/50
6/6 [==============================] - 0s 47ms/step - loss: 0.0692 - accuracy: 1.0000 - val_loss: 2.5169e-04 - val_accuracy: 1.0000
Epoch 46/50
6/6 [==============================] - 0s 45ms/step - loss: 0.0076 - accuracy: 1.0000 - val_loss: 5.4528e-04 - val_accuracy: 1.0000
Epoch 47/50
6/6 [==============================] - 0s 46ms/step - loss: 0.0121 - accuracy: 1.0000 - val_loss: 7.5272e-04 - val_accuracy:

Epoch 36/50
2/2 [==============================] - 0s 36ms/step - loss: 0.0236 - accuracy: 1.0000 - val_loss: 0.0017 - val_accuracy: 1.0000
Epoch 37/50
2/2 [==============================] - 0s 34ms/step - loss: 0.0022 - accuracy: 1.0000 - val_loss: 0.0015 - val_accuracy: 1.0000
Epoch 38/50
2/2 [==============================] - 0s 38ms/step - loss: 0.0204 - accuracy: 1.0000 - val_loss: 8.5982e-04 - val_accuracy: 1.0000
Epoch 39/50
2/2 [==============================] - 0s 34ms/step - loss: 0.0720 - accuracy: 1.0000 - val_loss: 5.9271e-04 - val_accuracy: 1.0000
Epoch 40/50
2/2 [==============================] - 0s 33ms/step - loss: 0.0154 - accuracy: 1.0000 - val_loss: 6.7104e-04 - val_accuracy: 1.0000
Epoch 41/50
2/2 [==============================] - 0s 33ms/step - loss: 0.0026 - accuracy: 1.0000 - val_loss: 8.9297e-04 - val_accuracy: 1.0000
Epoch 42/50
2/2 [==============================] - 0s 35ms/step - loss: 0.0021 - accuracy: 1.0000 - val_loss: 0.0011 - val_accuracy: 1.0000
Epoc

1/1 [==============================] - 0s 23ms/step - loss: 0.0105 - accuracy: 1.0000 - val_loss: 8.4573e-05 - val_accuracy: 1.0000
Epoch 31/50
1/1 [==============================] - 0s 23ms/step - loss: 6.3320e-04 - accuracy: 1.0000 - val_loss: 4.9470e-05 - val_accuracy: 1.0000
Epoch 32/50
1/1 [==============================] - 0s 25ms/step - loss: 1.6510e-05 - accuracy: 1.0000 - val_loss: 3.5285e-05 - val_accuracy: 1.0000
Epoch 33/50
1/1 [==============================] - 0s 23ms/step - loss: 3.5862e-04 - accuracy: 1.0000 - val_loss: 2.8729e-05 - val_accuracy: 1.0000
Epoch 34/50
1/1 [==============================] - 0s 24ms/step - loss: 2.9098e-04 - accuracy: 1.0000 - val_loss: 2.5928e-05 - val_accuracy: 1.0000
Epoch 35/50
1/1 [==============================] - 0s 23ms/step - loss: 2.1632e-04 - accuracy: 1.0000 - val_loss: 2.4378e-05 - val_accuracy: 1.0000
Epoch 36/50
1/1 [==============================] - 0s 23ms/step - loss: 4.0232e-05 - accuracy: 1.0000 - val_loss: 2.3544e-05 - v

In [21]:
plt.figure(figsize=(14, 6))
plt.rc('font', size=18)

plt.plot(history.history["accuracy"])
plt.plot(history.history["val_accuracy"])
plt.ylabel("accuracy")
plt.xlabel("epoch")
plt.legend(["train_accuracy", "val_accuracy"])
plt.show()

NameError: name 'history' is not defined

<Figure size 1008x432 with 0 Axes>